# Notebook to test the RSI indicator

Latest version: 2024-08-16  
Author: MvS

## Description

sources:

- [Investopedia](https://www.investopedia.com/terms/r/rsi.asp)
- [Medium](https://medium.com/@manreddyr/calculating-relative-strength-index-using-pandas-80f7b4f2653d)
- [How to use and tweak](https://thepatternsite.com/RSI.html)

## Result




In [2]:
import yfinance as yf
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from importlib import reload  # Python 3.4+
import utils.indicator_utils as idc

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s")

periods = [14]

dt_end = datetime.datetime.today()
dt_data_start = dt_end - datetime.timedelta(days=max(periods) * 10)

# stock = "FROG"
# stock = "TSLA"
# stock = "NVDA"
stock = "SAP.TO"

try:
    # Grab sufficient stock data for averaging SMAs
    load_df = yf.download(
        f"{stock}",
        start=dt_data_start.strftime('%Y-%m-%d'),
        end=dt_end.strftime('%Y-%m-%d'),
        progress=False,
    )

    assert load_df.shape[1] == 6 and load_df.shape[0] > max(periods)

except AssertionError:
    logging.info(f"Download failed for symbol {stock}.  Skipping...")


In [ ]:
reload(idc)

sym_df = load_df.copy()

# Compute the RSI
for period in periods:
    sym_df[f"RSI_{period:03}"] = idc.wilder_rsi(close=sym_df['Close'], length=period)


dt_start = sym_df.index[0]

logging.info(
    f"""Calculating RSI indicator for {periods} window length starting from:
  {dt_start.strftime('%Y-%m-%d')} and ending on {dt_end.strftime('%Y-%m-%d')}
  and containing {sym_df.shape} data points."""
)


# Define the columns and their styles
columns = ['Close', ]
colors = ['darkgreen', ]
linestyles = ['-', ]
markers = [None, ]

try:
    # Plot each column separately
    ax = None
    for col, color, style, marker in zip(columns, colors, linestyles, markers):
        ax = sym_df[col].plot(color=color, linestyle=style, marker=marker, ax=ax)
    ax.legend(loc='upper left')
    # Plot RSI on secondary axis
    second_ax = ax.twinx()
    sym_df[f"RSI_{period:03}"].plot(color='purple', linestyle='--', ax=second_ax)
    second_ax.legend(loc='upper right')

except TypeError:
    logging.info(f"Error during plotting")
